In [1]:
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
import tensorflow as tf

In [3]:
import numpy as np

In [4]:
import random

In [5]:
config=tf.ConfigProto()
config.gpu_options.allow_growth = True

In [6]:
tf.set_random_seed(777)  # reproducibility

In [7]:
# parameters
log_dir = './logs/ML_Lab_10-02-NN_512x5'
learning_rate = 0.001
batch_size = 100
num_epochs = 50

In [8]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [9]:
print(mnist.train.num_examples)

55000


In [10]:
nb_classes = 10

In [11]:
X = tf.placeholder(shape=[None, 784], dtype=tf.float32, name='X')

In [12]:
Y = tf.placeholder(shape=[None, nb_classes], dtype=tf.int32, name='Y')

In [13]:
with tf.variable_scope('layer1') as scope:
    W1 = tf.Variable(tf.random_normal(shape=[784, 512]), shape=[784, 512], dtype=tf.float32, name='weight1')
    b1 = tf.Variable(tf.random_normal(shape=[512]), shape=[512], dtype=tf.float32, name='bias1')
    L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

In [14]:
with tf.variable_scope('layer2') as scope:
    W2 = tf.Variable(tf.random_normal(shape=[512, 512]), shape=[512, 512], dtype=tf.float32, name='weight2')
    b2 = tf.Variable(tf.random_normal(shape=[512]), shape=[512], dtype=tf.float32, name='bias2')
    L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

In [15]:
with tf.variable_scope('layer3') as scope:
    W3 = tf.Variable(tf.random_normal(shape=[512, 512]), shape=[512, 512], dtype=tf.float32, name='weight3')
    b3 = tf.Variable(tf.random_normal(shape=[512]), shape=[512], dtype=tf.float32, name='bias3')
    L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)

In [16]:
with tf.variable_scope('layer4') as scope:
    W4 = tf.Variable(tf.random_normal(shape=[512, 512]), shape=[512, 512], dtype=tf.float32, name='weight4')
    b4 = tf.Variable(tf.random_normal(shape=[512]), shape=[512], dtype=tf.float32, name='bias4')
    L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)

In [17]:
with tf.variable_scope('layer5') as scope:
    W5 = tf.Variable(tf.random_normal(shape=[512, 10]), shape=[512, 10], dtype=tf.float32, name='weight5')
    b5 = tf.Variable(tf.random_normal(shape=[10]), shape=[10], dtype=tf.float32, name='bias5')
    hypothesis = tf.matmul(L4, W5) + b5

In [18]:
with tf.variable_scope('cost'):
    cost_i = tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y)
    cost = tf.reduce_mean(cost_i)

In [19]:
with tf.variable_scope('Train'):
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [20]:
with tf.variable_scope('accuracy') as scope:
    is_correct = tf.equal(tf.argmax(hypothesis, 1), tf.arg_max(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

Instructions for updating:
Use `tf.math.argmax` instead


In [21]:
with tf.variable_scope('tensorboard') as scope:
    overall_cost     = tf.placeholder(shape=None, dtype=tf.float32, name='overall_cost')
    overall_accuracy = tf.placeholder(shape=None, dtype=tf.float32, name='overall_accuracy')
    tf.summary.scalar('cost', overall_cost)
    tf.summary.scalar('accuracy', overall_accuracy)

In [22]:
with tf.Session(config=config) as sess:
    # tensorboard --logdir=./logs/Deep_NN
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter(log_dir)
    writer.add_graph(sess.graph)
    
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())
    # Training cycle
    for epoch in range(num_epochs):
        _overall_cost = 0
        total_batch = int(mnist.train.num_examples / batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            cost_val, _ = sess.run([cost, train], feed_dict={X: batch_xs, Y: batch_ys})
            _overall_cost += cost_val / total_batch
        
        _overall_accuracy = 0
        total_batch = int(mnist.test.num_examples / batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.test.next_batch(batch_size)
            accuracy_val = sess.run(accuracy, feed_dict={X: batch_xs, Y: batch_ys})
            _overall_accuracy += accuracy_val / total_batch
            
        summary = sess.run(merged_summary, feed_dict={overall_cost: _overall_cost, overall_accuracy: _overall_accuracy})
        writer.add_summary(summary, global_step=(epoch))
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(_overall_cost), 'accuracy =', '{:.9f}'.format(_overall_accuracy))

Epoch: 0001 cost = 42153.306005859 accuracy = 0.893400001
Epoch: 0002 cost = 9297.538738292 accuracy = 0.918600000
Epoch: 0003 cost = 4755.556309450 accuracy = 0.921400004
Epoch: 0004 cost = 2665.744743342 accuracy = 0.931700002
Epoch: 0005 cost = 1541.931664763 accuracy = 0.935100004
Epoch: 0006 cost = 1034.569558746 accuracy = 0.940300001
Epoch: 0007 cost = 646.388983336 accuracy = 0.933800000
Epoch: 0008 cost = 616.195537167 accuracy = 0.941400002
Epoch: 0009 cost = 639.835489396 accuracy = 0.941800001
Epoch: 0010 cost = 568.460299317 accuracy = 0.947600004
Epoch: 0011 cost = 455.434816737 accuracy = 0.949100000
Epoch: 0012 cost = 495.901792905 accuracy = 0.951200001
Epoch: 0013 cost = 447.417832991 accuracy = 0.948700002
Epoch: 0014 cost = 419.111195151 accuracy = 0.952100002
Epoch: 0015 cost = 402.363639008 accuracy = 0.952900003
Epoch: 0016 cost = 441.454856366 accuracy = 0.956000001
Epoch: 0017 cost = 319.807500028 accuracy = 0.956500003
Epoch: 0018 cost = 346.175901138 accuracy